Import gensim to use Word2Vec and Doc2Vec, and logging 

In [ ]:
import gensim, logging

Add logging configuration

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Load a pre-built Word2Vec model provided by Google

In [ ]:
gmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

Printing the vector of the word cat

In [ ]:
# gmodel['cat']
len(gmodel["cat"]) # 300

Getting the similarity between some words

In [ ]:
gmodel.similarity("cat", "dog")

Setup the Doc2Vec model

In [ ]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

Define a function to format the data for the TaggedDocument

In [ ]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) # strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remove apostrophes
    sent = re.sub(r'\W', ' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated spaces
    sent = sent.strip()
    return sent.split()

In [ ]:
# unsupervised training data
import re
import os
unpup_sentences = []

# source: https://ai.stanford.edu/~amaas/data/sentiment, data from IMDB
for dirname in ['train/pos', "train/neg", "train/unsup", "test/pos", "test/neg"]:
    for fname in sorted(os.listdir("aclImdb/"+dirname)):
        if fname[-4:] == ".txt":
            with open("aclImdb/"+dirname+"/"+fname, encoding="UTF-8") as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname+"/"+fname]))

# source: http://cs.cornell.edu/people/pabo/movie-review-data/
for dirname in ["txt_sentoken/pos", "txt_sentoken/neg"]:
    for fname in sorted(os.listdir(dirname)):
        if fname[-4:] == ".txt":
            with open(dirname + "/" + fname, encoding="UTF-8") as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s/%s-%d" % (dirname, fname, i)]))

# source: https://nlp.stanford.edu/sentiment/, data from Rottn Tomatoes
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding="UTF-8") as f:
    for i, line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocument(words, ["rt-%d" % i]))